In [1]:
import numpy as np

In [2]:
from nerf.network import NeTFMLP4 as NeRFNetwork 

In [3]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [10]:
opt = AttrDict(dict(
    bound=4,
    bg_radius=2
))

In [11]:
model = NeRFNetwork(
    encoding="hashgrid",
    bound=opt.bound,
    cuda_ray=False,
    density_scale=1,
    min_near=0.2,
    density_thresh=10,
    bg_radius=opt.bg_radius,
)


In [12]:
import torch

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [14]:
checkpoint_dict = torch.load("trial_nerf_resnet/checkpoints/ngp_ep0348.pth", map_location=device)

In [15]:
missing_keys, unexpected_keys = model.load_state_dict(checkpoint_dict['model'], strict=False)

In [18]:
bound = 1
N = 512
x_ = np.linspace(-bound, bound, N)
y_ = np.linspace(-bound, bound, N)
z_ = np.linspace(-bound, bound, N)
x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')
X = np.stack([x, y, z], axis=-1).reshape((-1, 3))

In [19]:
X.shape

(134217728, 3)

In [20]:
bound = opt.bound
N = 512
alpha_ = np.linspace(0, 2 * np.pi, N)
beta_ = np.linspace(0, 2 * np.pi, N)
alpha, beta = np.meshgrid(alpha_, beta_, indexing='ij')
x = np.cos(alpha) * np.cos(beta);
z = np.sin(alpha) * np.cos(beta);
y = np.sin(beta);
D = np.stack([x, y, z], axis=-1).reshape((-1, 3))

In [21]:
D.shape

(262144, 3)

In [22]:
model = model.to(device)

In [23]:
torch.tensor(X[:10].astype(np.float16)).to(device).dtype

torch.float16

In [24]:
batch = 2**16

In [25]:
from tqdm.auto import tqdm

In [26]:
results = []
with torch.cuda.amp.autocast(enabled=True):
    for k in tqdm(range(int(X.shape[0] / batch))):
        h = model.encoder(torch.tensor(X[k * batch:(k + 1) * batch].astype(np.float32)).to(device))
        for l in model.sigma_net:
            h = l(h)
            
        results.append(h.detach().cpu().numpy())

  0%|          | 0/2048 [00:00<?, ?it/s]

In [27]:
X.shape

(134217728, 3)

In [28]:
np.vstack(results).shape

(134217728, 8)

In [29]:
results_pos = np.vstack(results).reshape([512, 512, 512, -1])

In [30]:
 with open("cache_pos.npy", "wb") as f:
    np.save(f, results_pos, allow_pickle=False)

In [31]:
 with open("cache_pos_32.npy", "wb") as f:
    np.save(f, results_pos.astype(np.float32), allow_pickle=False)

In [32]:
 with open("cache_pos_32_p1.npy", "wb") as f:
    np.save(f, results_pos[:, :, :, :4].astype(np.float32), allow_pickle=False)

In [33]:
 with open("cache_pos_32_p2.npy", "wb") as f:
    np.save(f, results_pos[:, :, :,4:].astype(np.float32), allow_pickle=False)

In [37]:
results_dir = []
with torch.cuda.amp.autocast(enabled=True):
    for k in tqdm(range(int(D.shape[0] / batch))):
        h = model.encoder_dir(torch.tensor(D[k * batch:(k + 1) * batch].astype(np.float32)).to(device))
        for l in model.gamma_net:
            h = l(h)
            
        results_dir.append(h.detach().cpu().numpy())

  0%|          | 0/4 [00:00<?, ?it/s]

In [38]:
D.shape[0]

262144

In [39]:
np.vstack(results_dir).shape

(262144, 8)

In [40]:
results_dir_ = np.vstack(results_dir).reshape([512, 512, -1])

In [41]:
results_dir_.shape

(512, 512, 8)

In [29]:
 with open("cache_dir.npy", "wb") as f:
    np.save(f, results_dir_, allow_pickle=False)

In [30]:
 with open("cache_dir_32.npy", "wb") as f:
    np.save(f, results_dir_.astype(np.float32), allow_pickle=False)

In [43]:
 with open("cache_dir_32_p1.npy", "wb") as f:
    np.save(f, results_dir_[:, :, :4].astype(np.float32), allow_pickle=False)

In [44]:
 with open("cache_dir_32_p2.npy", "wb") as f:
    np.save(f, results_dir_[:, :, 4:].astype(np.float32), allow_pickle=False)